In [1]:
import secretflow as sf

import sys
sys.path.append("/home/yejj/GradExperiments/chapter3")

sf.shutdown()
#init 10 nodes and 1 server
nodes = [f"party{i}" for i in range(10)]
nodes.append('server')

# init nodes
sf.init(nodes, address="local")
# init PYU device
devices = {}
for i in range(10):
    devices[i] = sf.PYU(f"party{i}")
# alice, bob, agg_server = sf.PYU("alice"), sf.PYU("bob"), sf.PYU("server")
agg_server = sf.PYU("server")

from secretflow.utils.simulation.datasets import load_mnist

(x_train, y_train), (x_test, y_test) = load_mnist(parts=[devices[i] for i in range(10)], normalized_x=True, categorical_y=True)
x_train.partition_shape()

2024-02-22 21:57:02,076	INFO worker.py:1538 -- Started a local Ray instance.
/home/yejj/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/yejj/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


{PYURuntime(party0): (6000, 28, 28),
 PYURuntime(party1): (6000, 28, 28),
 PYURuntime(party2): (6000, 28, 28),
 PYURuntime(party3): (6000, 28, 28),
 PYURuntime(party4): (6000, 28, 28),
 PYURuntime(party5): (6000, 28, 28),
 PYURuntime(party6): (6000, 28, 28),
 PYURuntime(party7): (6000, 28, 28),
 PYURuntime(party8): (6000, 28, 28),
 PYURuntime(party9): (6000, 28, 28)}

In [2]:
from secretflow.ml.nn import FLModel
from secretflow.security.aggregation import PlainAggregator
from utils.model import create_model
num_classes = 10
input_shape = (28, 28, 1)
model = create_model(input_shape, num_classes, "lenet")

plain_aggregator = PlainAggregator(agg_server)

fed_plain_model = FLModel(server=agg_server,
                    device_list=[devices[i] for i in range(10)],
                    model=model,
                    aggregator=plain_aggregator,
                    strategy="fed_avg_w",
                    backend = "tensorflow")

from secretflow.ml.nn.callbacks.early_stopping import EarlyStoppingEpoch
earlystop_callback = EarlyStoppingEpoch(
		monitor='val_loss', min_delta=0.1, mode='min',
        patience=2)

import time
start_time = time.time()
history = fed_plain_model.fit(x_train,
                    y_train,
                    validation_data=(x_test, y_test),
                    epochs=2,
                    sampler_method="batch",
                    batch_size=128,
                    aggregate_freq=1,
                    callbacks=earlystop_callback)
plain_time = time.time() - start_time
# print("cost time : ", time.time()-start_time)
history

2024-02-22 21:57:08.405087: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-22 21:57:09.500198: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-22 21:57:09.500317: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-02-22 21:57:09.500327: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

(pid=17707) 2024-02-22 21:57:15.253228: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=17667) 2024-02-22 21:57:15.291518: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=17870) 2024-02-22 21:57:15.649766: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=18151) 2024-02-22 21:57:16.172177: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No suc

Epoch 1/2


Train Processing: :   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 2/2


Train Processing: :  98%|█████████▊| 46/47 [00:07<00:00,  6.26it/s, {'loss': 0.4367702, 'accuracy': 0.8701286, 'val_loss': 0.30069393, 'val_accuracy': 0.9132}]


{'global_history': {'loss': [1.402708, 0.4367702],
  'accuracy': [0.6202, 0.8701286],
  'val_loss': [0.48212877, 0.30069393],
  'val_accuracy': [0.8561, 0.9132]},
 'local_history': {'party0_loss': [1.4011714, 0.4304856],
  'party0_accuracy': [0.61266667, 0.8747143],
  'party0_val_loss': [0.5349864, 0.32775438],
  'party0_val_accuracy': [0.826, 0.899],
  'party1_loss': [1.3837107, 0.45385352],
  'party1_accuracy': [0.62866664, 0.8642857],
  'party1_val_loss': [0.6436127, 0.43075618],
  'party1_val_accuracy': [0.814, 0.874],
  'party2_loss': [1.4322951, 0.48986185],
  'party2_accuracy': [0.60033333, 0.84985715],
  'party2_val_loss': [0.5886445, 0.3731605],
  'party2_val_accuracy': [0.831, 0.888],
  'party3_loss': [1.4045137, 0.434759],
  'party3_accuracy': [0.6128333, 0.8742857],
  'party3_val_loss': [0.61489105, 0.4006433],
  'party3_val_accuracy': [0.829, 0.879],
  'party4_loss': [1.4165958, 0.4641799],
  'party4_accuracy': [0.6281667, 0.86414284],
  'party4_val_loss': [0.5900066, 0.38